In [29]:
import findspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import count, countDistinct, approx_count_distinct
from pyspark.sql.functions import min, max, col
from pyspark.sql.functions import sum, sum_distinct, avg
from pyspark.sql.functions import desc

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext

## 1. Agregaciones

La realización de análisis interesante sobre Big Data generalmente implica algún tipo de agregación para resumir los datos con el fin de extraer patrones, conocimientos o simplemente generar informes resumidos.

Las agregaciones generalmente requieren de alguna forma de agrupación, ya sea en todo el conjunto de datos o en una o más columnas, y luego aplican funciones de agregación como sumar, contar o promediar a cada grupo.

Spark proporciona muchas funciones de agregación de uso común, así como la capacidad de agregar los valores de una colección que luego se pueden analizar más a fondo.

La agrupación de filas se puede realizar en diferentes niveles y spark admite los siguientes niveles:

* Tratar un DF como un grupo.

* Dividir un DF en varios grupos utilizando una o más columnas y realizar una o más agregaciones en cada uno de estos grupos.

* Dividir un DF en varias ventanas y realizar una media móvil, una suma acumulativa o una clasificación.

En spark todas las agregaciones se realizan a través de funciones.\
Las funciones de agregación están diseñadas para realizar la agregación en un conjunto de filas, yasea que ese conjunto de filas consista en todas las filas o en un subgrupo de filas en un DF.

In [2]:
df_vuelos = spark.read.parquet('./data/flights/vuelos.parquet')

In [4]:
df_vuelos.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [6]:
df_vuelos.show(5, truncate=False)

23/07/25 08:45:18 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

### 1-1. Funciones count, countDistinct y aprox_count_distinct

In [7]:
df = spark.read.parquet('./data/flights/dataframe.parquet')

In [8]:
df.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- color: string (nullable = true)
 |-- cantidad: long (nullable = true)



In [9]:
df.show()

+------+-----+--------+
|nombre|color|cantidad|
+------+-----+--------+
|  Jose| azul|    1900|
|  null| null|    1700|
|  null| rojo|    1300|
|  Juan| rojo|    1500|
+------+-----+--------+



* **count** es una agregación que nos permite saber la cantidad de elementos de un grupo. 

In [11]:
# Vamos a contar la cantidad de nombres y de colores del DF. Cuenta todos aquellos valores diferentes de null que se encuentren en la columna.

df.select(
    count('nombre').alias('conteo_nombre'),
    count('color').alias('conteo_color')
).show()

+-------------+------------+
|conteo_nombre|conteo_color|
+-------------+------------+
|            2|           3|
+-------------+------------+



In [12]:
# Si queremos que también incluya también los null, que cuente todas las filas, le pasamos '*' como parámetro.

df.select(
    count('nombre').alias('conteo_nombre'),
    count('color').alias('conteo_color'),
    count('*').alias('conteo_general')
).show()

+-------------+------------+--------------+
|conteo_nombre|conteo_color|conteo_general|
+-------------+------------+--------------+
|            2|           3|             4|
+-------------+------------+--------------+



* **countDistinct** admite los nulos y cuenta los valores diferentes.

In [15]:
# Contamos los colores que podemos encontrar en la columna color

df.select(
    countDistinct('color').alias('colores_dif')
).show()

+-----------+
|colores_dif|
+-----------+
|          2|
+-----------+



* **approx_count_distinct** realiza un recuento aproximado de la cantidad total de valores. Hay que tener en cuenta que contar el número exacto de elementos únicos en cada grupo en un gran conjunto de datos es una operación costosa y que se requiere mucho tiempo, en algunos casos de uso es suficiente a veces tener un recuento único aproximado.

In [17]:
df_vuelos.select(
    countDistinct('AIRLINE'),
    approx_count_distinct('AIRLINE')
).show()


+-----------------------+------------------------------+
|count(DISTINCT AIRLINE)|approx_count_distinct(AIRLINE)|
+-----------------------+------------------------------+
|                     14|                            13|
+-----------------------+------------------------------+



### 1-2. Funciones min y max

In [20]:
# Vamos a hallar el tiempo mínimo y máximo que estuvieron los aviones en el aire

df_vuelos.select(
    min('AIR_TIME').alias('menor_timepo'),
    max('AIR_TIME').alias('mayor_tiempo')
).show()

+------------+------------+
|menor_timepo|mayor_tiempo|
+------------+------------+
|           7|         690|
+------------+------------+



In [21]:
# Vamos a ver el tiempo mínimo y máximo de retraso en los vuelos

df_vuelos.select(
    min('AIRLINE_DELAY'),
    max('AIRLINE_DELAY')
).show()

+------------------+------------------+
|min(AIRLINE_DELAY)|max(AIRLINE_DELAY)|
+------------------+------------------+
|                 0|              1971|
+------------------+------------------+



### 1-3. Funciones sum, sum_distinct y avg

* **sum** calcula la suma de los valores de una columna numérica

In [23]:
# Veamos la distancia total recorrida en todos los vuelos

df_vuelos.select(
    sum('DISTANCE').alias('sum_dis')
).show()

+----------+
|   sum_dis|
+----------+
|4785357409|
+----------+



* **sum_distinc** suma sólo los valores distintos de una columna

In [27]:
df_vuelos.select(
    sum_distinct('DISTANCE').alias('sum_dis_dif')
).show()

+-----------+
|sum_dis_dif|
+-----------+
|    1442300|
+-----------+



* **avg** calcula el valor promedio de una columna numérica.

In [28]:
# Veamos la media de tiempo en el aire

df_vuelos.select(
    avg('AIR_TIME').alias('promedio_aire'),
    (sum('AIR_TIME') / count('AIR_TIME')).alias('prom_manual')
).show()


+------------------+------------------+
|     promedio_aire|       prom_manual|
+------------------+------------------+
|113.51162809012519|113.51162809012519|
+------------------+------------------+



### 1-4. Agregación con agrupación

Las agregaciones generalmente se realizan en conjuntos de datos que contienen una o más columnas categóricas que tienen una baja cardinalidad. Por ejemplo, algunos valores categóricos son el sexo, la edad, nombre de la ciudad o nombre del país.

Las agregaciones se realizan a través de funciones similares a las estudiadas anteriormente. Sin embargo, en lugar de realizar la agregación en el DF, realizaremos la agregación en cada uno de los subgrupos dentro de un DF.

Realizar la agregación con agrupación es un proceson de dos pasos:

1. realizar la agrupación mediante la transformación groupBy(col1, col2, ...) de las columnas que deseemos agrupar y ahí es donde se especifica por qué columnas debemos agrupar las filas. A diferencia de otras transformaciones que devuelven un DF, la transformación groupBy devuelve una instancia de la clase relacional grouped dataset a la que luego puedes aplicar una o más funciones de agregación.

2. aplicar las funciones de agregación deseadas.

Debemos tener en cuenta que la clase del relacional grouped dataset, que es lo que nos devuelve el groupBy, proporciona un conjunto de funciones estándares de agregación que puede aplicar a cada subgrupo. Dentro de esta funciones tenemos el **avg**, el **count**, el **mean**, el **max**, el **min** o el **sum** entre otras.

In [31]:
# Veamos cuántos aeropuertos de origen tenemos. Agruparemos por los aeropuertos de origen y contaremos cuántos vuelos salieron de cada uno de ellos

(df_vuelos.groupBy('ORIGIN_AIRPORT')
    .count()
    .orderBy(desc('count'))
).show()


+--------------+------+
|ORIGIN_AIRPORT| count|
+--------------+------+
|           ATL|346836|
|           ORD|285884|
|           DFW|239551|
|           DEN|196055|
|           LAX|194673|
|           SFO|148008|
|           PHX|146815|
|           IAH|146622|
|           LAS|133181|
|           MSP|112117|
|           MCO|110982|
|           SEA|110899|
|           DTW|108500|
|           BOS|107847|
|           EWR|101772|
|           CLT|100324|
|           LGA| 99605|
|           SLC| 97210|
|           JFK| 93811|
|           BWI| 86079|
+--------------+------+
only showing top 20 rows



In [33]:
# Ahora vamos a agrupar por el aeropuerto origen y el aeropuerto destino, para contar cuáles fueron los trayectos más frecuentados

(df_vuelos.groupBy('ORIGIN_AIRPORT', 'DESTINATION_AIRPORT')
    .count()
    .orderBy(desc('count'))
).show()

+--------------+-------------------+-----+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|count|
+--------------+-------------------+-----+
|           SFO|                LAX|13744|
|           LAX|                SFO|13457|
|           JFK|                LAX|12016|
|           LAX|                JFK|12015|
|           LAS|                LAX| 9715|
|           LGA|                ORD| 9639|
|           LAX|                LAS| 9594|
|           ORD|                LGA| 9575|
|           SFO|                JFK| 8440|
|           JFK|                SFO| 8437|
|           OGG|                HNL| 8313|
|           HNL|                OGG| 8282|
|           LAX|                ORD| 8256|
|           ATL|                LGA| 8234|
|           LGA|                ATL| 8215|
|           ATL|                MCO| 8202|
|           MCO|                ATL| 8202|
|           SFO|                LAS| 7995|
|           ORD|                LAX| 7941|
|           LAS|                SFO| 7870|
+----------

### 1-5. Varias agregaciones por grupo

La clase del relacional grouped dataset proporciona una función llamada **agg** que toma una o más expresiones de columna, lo que significa que puede usar cualquiera de las funciones de agregación estudiadas en los puntos anteriores.

In [34]:
# Vamos a agrupar por el aeropuerto origen y agregaremos el conteo del tiempo que estuvieron esos aviones en el aire y el min y max de ese tiempo

df_vuelos.groupBy('ORIGIN_AIRPORT').agg(
    count('AIR_TIME').alias('tiempo_aire'),
    min('AIR_TIME').alias('min'),
    max('AIR_TIME').alias('max')
).orderBy(desc('tiempo_aire')).show()

+--------------+-----------+---+---+
|ORIGIN_AIRPORT|tiempo_aire|min|max|
+--------------+-----------+---+---+
|           ATL|     343506| 15|614|
|           ORD|     276554| 13|571|
|           DFW|     232647| 11|534|
|           DEN|     193402| 12|493|
|           LAX|     192003| 14|409|
|           PHX|     145552| 19|444|
|           SFO|     145491|  8|389|
|           IAH|     144019| 15|524|
|           LAS|     131937| 25|429|
|           MSP|     111055| 14|537|
|           SEA|     110178| 17|412|
|           MCO|     109532| 25|395|
|           DTW|     106992| 15|341|
|           BOS|     104804| 16|432|
|           CLT|      99052| 17|379|
|           EWR|      98341| 21|683|
|           SLC|      96505| 18|419|
|           LGA|      94834| 19|311|
|           JFK|      91663| 29|690|
|           BWI|      84329| 19|398|
+--------------+-----------+---+---+
only showing top 20 rows



In [35]:
# Ahora vamos a agrupar por el mes, agregamos el conteo de vuelos que llegaron con retraso y vemos el promedio de distancia de esos vuelos

df_vuelos.groupBy('MONTH').agg(
    count('ARRIVAL_DELAY').alias('conteo_de_retrasos'),
    avg('DISTANCE').alias('prom_dist')
).orderBy(desc('conteo_de_retrasos')).show()

+-----+------------------+-----------------+
|MONTH|conteo_de_retrasos|        prom_dist|
+-----+------------------+-----------------+
|    7|            514384|841.4772794487611|
|    8|            503956|834.8244276603413|
|    6|            492847|835.6302716626612|
|    3|            492138|816.0553268611494|
|    5|            489641|823.3230588760807|
|   10|            482878|816.4436127652134|
|    4|            479251|817.0060476016745|
|   12|            469717|837.8018926194103|
|   11|            462367|820.2482434846529|
|    9|            462153|815.8487523282274|
|    1|            457013|803.2612794913696|
|    2|            407663| 800.785449834689|
+-----+------------------+-----------------+



### 1-6. Agregación con pivote

Permite agregar los resultados de las funciones por cada uno de los valores diferentes de una columna.\
Si le especificamos una lista de valores diferentes para la columna pivote, lo que va a hacer en realidad es acelerar el proceso de rotación de esta columna, el proceso de pivoteo. De lo contrario, spark va a dedicar un poco de tiempo y de esfuerzo a encontrar una lista de los valores distintos por sí solo.

In [36]:
df_estudiantes = spark.read.parquet('./data/estudiantes.parquet')

df_estudiantes.show()

+------+----+----+----------+
|nombre|sexo|peso|graduacion|
+------+----+----+----------+
|  Jose|   M|  80|      2000|
| Hilda|   F|  50|      2000|
|  Juan|   M|  75|      2000|
| Pedro|   M|  76|      2001|
|Katia+|   F|  65|      2001|
+------+----+----+----------+



In [37]:
# Vamos a ver el peso promedio por sexo en cada año de graduación utilizando pivot

df_estudiantes.groupBy('graduacion').pivot('sexo').agg(avg('peso')).show()

+----------+----+----+
|graduacion|   F|   M|
+----------+----+----+
|      2001|65.0|76.0|
|      2000|50.0|77.5|
+----------+----+----+



In [38]:
# Además del promedio de peso, vamos a obtener el mínimo y el máximo

df_estudiantes.groupBy('graduacion').pivot('sexo').agg(avg('peso'), min('peso'), max('peso')).show()

+----------+-----------+-----------+-----------+-----------+-----------+-----------+
|graduacion|F_avg(peso)|F_min(peso)|F_max(peso)|M_avg(peso)|M_min(peso)|M_max(peso)|
+----------+-----------+-----------+-----------+-----------+-----------+-----------+
|      2001|       65.0|         65|         65|       76.0|         76|         76|
|      2000|       50.0|         50|         50|       77.5|         75|         80|
+----------+-----------+-----------+-----------+-----------+-----------+-----------+



In [39]:
# Podemos tomar sólo algunos de los elementos presentes en la columna sobre la que pivotemos, que le pasaremos en forma de lista

df_estudiantes.groupBy('graduacion').pivot('sexo', ['M']).agg(avg('peso'), min('peso'), max('peso')).show()

+----------+-----------+-----------+-----------+
|graduacion|M_avg(peso)|M_min(peso)|M_max(peso)|
+----------+-----------+-----------+-----------+
|      2001|       76.0|         76|         76|
|      2000|       77.5|         75|         80|
+----------+-----------+-----------+-----------+



In [40]:
# Lo mismo que en la celda anterior, pero para el el sexo femenino

df_estudiantes.groupBy('graduacion').pivot('sexo', ['F']).agg(avg('peso'), min('peso'), max('peso')).show()

+----------+-----------+-----------+-----------+
|graduacion|F_avg(peso)|F_min(peso)|F_max(peso)|
+----------+-----------+-----------+-----------+
|      2001|       65.0|         65|         65|
|      2000|       50.0|         50|         50|
+----------+-----------+-----------+-----------+

